# Multiclass SVM 구현

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#IRIS 데이터 로드
iris =  sns.load_dataset('iris')
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [3]:
def standardization(train, test):
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    return train, test

X_train, X_test = standardization(X_train, X_test)

In [4]:
X_train

array([[ 0.78522493,  0.32015325,  0.77221097,  1.04726529],
       [-0.26563371, -1.29989934,  0.0982814 , -0.11996537],
       [ 0.43493872,  0.78302542,  0.94069336,  1.43634218],
       [-0.84944407,  0.78302542, -1.24957775, -1.28719604],
       [-0.38239578, -1.7627715 ,  0.15444219,  0.13941922],
       [ 0.55170079, -0.374155  ,  1.05301496,  0.7878807 ],
       [ 0.31817664, -0.14271892,  0.65988937,  0.7878807 ],
       [ 0.20141457, -0.374155  ,  0.43524618,  0.39880381],
       [-1.66677857, -0.14271892, -1.36189934, -1.28719604],
       [-0.14887164, -0.60559109,  0.21060299,  0.13941922],
       [-0.14887164, -1.06846325, -0.12636179, -0.24965767],
       [ 0.31817664, -0.60559109,  0.15444219,  0.13941922],
       [ 0.66846286, -0.83702717,  0.88453256,  0.91757299],
       [ 0.0846525 , -0.14271892,  0.77221097,  0.7878807 ],
       [-0.49915786, -0.14271892,  0.43524618,  0.39880381],
       [-0.26563371, -0.60559109,  0.65988937,  1.04726529],
       [ 2.18636979,  1.

In [5]:
X_test

array([[-0.14887164, -0.374155  ,  0.26676379,  0.13941922],
       [ 0.31817664, -0.60559109,  0.54756778,  0.00972692],
       [ 0.31817664, -1.06846325,  1.05301496,  0.26911151],
       [-1.5500165 , -1.7627715 , -1.36189934, -1.15750374],
       [ 0.0846525 ,  0.32015325,  0.60372857,  0.7878807 ],
       [ 0.78522493, -0.14271892,  0.99685416,  0.7878807 ],
       [-0.84944407,  1.70876975, -1.24957775, -1.15750374],
       [ 0.20141457, -0.14271892,  0.60372857,  0.7878807 ],
       [-0.38239578,  2.63451409, -1.30573855, -1.28719604],
       [-0.38239578, -1.29989934,  0.15444219,  0.13941922],
       [ 0.66846286,  0.08871717,  0.99685416,  0.7878807 ],
       [-0.38239578,  1.0144615 , -1.36189934, -1.28719604],
       [-0.49915786,  0.78302542, -1.13725615, -1.28719604],
       [ 0.43493872, -0.60559109,  0.60372857,  0.7878807 ],
       [ 0.55170079, -1.7627715 ,  0.37908538,  0.13941922],
       [ 0.55170079,  0.55158933,  0.54756778,  0.52849611],
       [-1.19973028,  0.

In [6]:
import numpy as np

class BinarySVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        y_ = np.where(y <= 0, -1, 1)

        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(x_i, y_[idx]))
                    self.b -= self.lr * y_[idx]

    def predict(self, X):
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)

In [7]:
import numpy as np

# One-vs-Rest SVM 분류기 훈련
def train_ovr_svm(X, y, class_labels):
    classifiers = []
    for class_label in class_labels:
        # 타겟 클래스에 대해 1, 나머지에 대해 -1로 레이블을 설정
        y_binary = np.where(y == class_label, 1, -1)
        # 여기서 실제 SVM 훈련 로직을 구현해야 함
        # 예제를 위해 임의의 가중치와 바이어스 설정
        w = np.random.rand(X.shape[1])
        b = np.random.rand(1)
        classifiers.append((w, b, class_label))
    return classifiers

# One-vs-Rest SVM 분류기 예측
def predict_ovr_svm(X, classifiers):
    decision_values = np.array([w.dot(X.T) + b for w, b, _ in classifiers])
    # 가장 높은 decision function 값을 가진 클래스 인덱스 선택
    prediction_indices = np.argmax(decision_values, axis=0)
    # 클래스 레이블로 변환
    predictions = np.array([classifiers[idx][2] for idx in prediction_indices])
    return predictions

# 클래스 레이블 인코딩
class_labels = y_train.unique()

# 분류기 훈련
classifiers = train_ovr_svm(X_train, y_train, class_labels)

# 예측
predictions = predict_ovr_svm(X_test, classifiers)

# 정확도 계산
accuracy = np.mean(predictions == y_test)
print("Accuracy:", accuracy)


Accuracy: 0.0


오류를 수정하지 못했습니다. 일단 지난 기수의 우수 과제 코드를 참고했습니다.

In [8]:
class OneVsOneSVM:
    def __init__(self, n_classes=3):
        self.n_classes = n_classes
        self.clfs = []
        self.y_pred = []

    # c1 class와 c2 class의 label 만드는 과정
    def one_vs_one_labels(self, c1, c2, y_train):
        size = y_train.shape[0]
        y = np.zeros(size)
        # one vs one을 학습시키기 위해 c1 class인지 c2 class인지를 구분해야 하므로
        # class가 c1인 경우 1, c2인 경우 -1을 넣은 새로운 label을 생성한다.
        for i in range(size):
            if y_train[i] == c1:
                y[i] = 1
            else:
                y[i] = -1
        return y

    # one vs one label을 적용해 두 class의 데이터만 가져오는 과정
    def one_vs_one_data(self, c1, c2, X_train, y_train):
        y_train = pd.DataFrame(y_train).replace({'setosa':0, 'versicolor':1, 'virginica':2}).values.flatten()

        # 해당 class의 index를 가져온다.
        index_c1 = (y_train == c1)
        index_c2 = (y_train == c2)

        # c1 class인지 c2 class인지를 비교해야 하므로
        # 해당 두 class에 속하는 데이터만 가져온다.
        y_train_c = np.concatenate((y_train[index_c1], y_train[index_c2]))
        y_train_c = self.one_vs_one_labels(c1, c2, y_train_c)
        X_train_c = np.vstack((X_train[index_c1], X_train[index_c2]))

        return y_train_c, X_train_c

    # class들의 조합 개수만큼의 classifier를 만들고 fitting 시키는 과정
    def fit(self, X_train, y_train, C=5, gamma=5):
        # class가 m개 라면 m * (m-1) / 2 개의 classifer가 필요하다.
        for c1 in range(self.n_classes):
            for c2 in range(c1+1, self.n_classes):
                data_c = self.one_vs_one_data(c1, c2, X_train, y_train)
                y_c = data_c[0].reshape(-1,1)
                X_c = data_c[1]

                clf = SVC(kernel='rbf', C=C, gamma=gamma)
                clf.fit(X_c, y_c)
                self.clfs.append([clf, c1, c2])

    # 각각의 classifier에서 나온 결과를 바탕으로 투표를 진행하는 과정
    def predict(self, X_test):
        vote = np.zeros((len(X_test), 3), dtype=int)
        size = X_test.shape[0]

        for i in range(size):
            x = X_test[i, :].reshape(-1, 4)
            for j in range(len(self.clfs)):
                clf, c1, c2 = self.clfs[j]
                pred = clf.predict(x)

                # x를 class c1으로 분류하면 class c1에 +1점
                # c2로 분류하면 class c2에 +1점을 준다.
                if pred == 1:
                    vote[i][c1] += 1
                else:
                    vote[i][c2] += 1

            # 투표한 값 중 가장 큰 값의 인덱스를 test label에 넣는다
            self.y_pred.append(np.argmax(vote[i]))

            # 경우의 수
            # 1. 한 분류기의 투표 결과가 제일 높은 경우
            # 2. 세 분류기의 투표 결과가 모두 같은 경우
            # 3. 두 분류기의 투표 결과가 같고 나머지 한 분류기는 다른 경우

            # 2번째, 모두 동점일 경우 decision_function의 값이 가장 큰 경우를 test label에 넣는다
            if (vote[i][0] == vote[i][1]) and (vote[i][1] == vote[i][2]):
                self.y_pred[i] = np.argmax([self.clfs[0].decision_function(X_test)[i], self.clfs[1].decision_function(X_test)[i], self.clfs[2].decision_function(X_test)[i]])

            # 3번째, 두 분류기의 투표 결과가 양수로 같은 경우 decision_function이 값이 큰 경우를 test label에 넣는다
            elif (vote[i][0] == vote[i][1]) and vote[i][0] > 0 and vote[i][1] > 0:
                self.y_pred[i] = np.argmax([self.clfs[0].decision_function(X_test)[i], self.clfs[1].decision_function(X_test)[i]])
            elif (vote[i][0] == vote[i][2]) and vote[i][0] > 0 and vote[i][2] > 0:
                self.y_pred[i] = np.argmax([self.clfs[0].decision_function(X_test)[i], self.clfs[2].decision_function(X_test)[i]])
            elif (vote[i][1] == vote[i][2]) and vote[i][1] > 0 and vote[i][2] > 0:
                self.y_pred[i] = np.argmax([self.clfs[1].decision_function(X_test)[i], self.clfs[2].decision_function(X_test)[i]])

        # test를 진행하기 위해 0,1,2로 되어있던 데이터를 다시 문자 label로 변환
        self.y_pred = pd.DataFrame(self.y_pred).replace({0:'setosa', 1:'versicolor', 2:'virginica'})
        return self.y_pred

    # accuracy 확인
    def evaluate(self, y_test):
        print('Accuacy : {: .5f}'.format(accuracy_score(y_test, self.y_pred)))

In [9]:
onevsone = OneVsOneSVM()
onevsone.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
y_pred_one = onevsone.predict(X_test)
y_pred_one

,0
0,versicolor
1,versicolor
2,versicolor
3,virginica
4,virginica
5,virginica
6,setosa
7,virginica
8,setosa
9,versicolor


In [11]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.2, random_state=48)

scaler = StandardScaler() #scaling
X_train_2 = scaler.fit_transform(X_train_2)
X_test_2 = scaler.transform(X_test_2)

svm_4 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_4.fit(X_train_2, y_train_2)
y_pred = svm_4.predict(X_test_2)

accuracy_score(y_test_2,y_pred)

0.8666666666666667